# Create a basic dashboard using Flask

## Step 1: Create the Project Structure
Create the following directory structure for your project:

my_dashboard_app/\
\
└──  app.py\
└──  templates/ \
└──└── base.html \
└──└── index.html \
└── static/ \
└──└── style.css

## Step 2: Create the Flask Application

Create a new Python file named app.py and add the following code:

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import io
import base64

app = Flask(__name__)

# Initialize the SQLite database
def init_db():
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            category TEXT NOT NULL,
            value REAL NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# Route for the home page with form to add data
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        category = request.form['category']
        value = request.form['value']
        
        conn = sqlite3.connect('data.db')
        c = conn.cursor()
        c.execute('INSERT INTO data (category, value) VALUES (?, ?)', (category, value))
        conn.commit()
        conn.close()
        
        return redirect(url_for('index'))
    
    return render_template('index.html')

# Route to display the dashboard with seaborn plots
@app.route('/dashboard')
def dashboard():
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    c.execute('SELECT category, value FROM data')
    data = c.fetchall()
    conn.close()
    
    # Convert data to a DataFrame
    import pandas as pd
    df = pd.DataFrame(data, columns=['Category', 'Value'])
    
    # Create a seaborn plot
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Category', y='Value', data=df)
    
    # Save the plot to a PNG image in memory
    img = io.BytesIO()
    plt.savefig(img, format='png')
    img.seek(0)
    
    # Encode the image to base64 string
    plot_url = base64.b64encode(img.getvalue()).decode()
    
    return render_template('dashboard.html', plot_url=plot_url)

if __name__ == '__main__':
    init_db()
    app.run(debug=True)

## Step 3: Create the template for the index page

Create a file named index.html in the templates directory with the following content:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Daily Activity Data Tracker</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h1>Activity Data Entry</h1>
    <form method="POST">
        <label for="category">Activity:</label>
        <input type="text" id="category" name="category" required>
        <br>
        <label for="value">Minutes:</label>
        <input type="number" id="value" name="value" step="1" required>
        <br>
        <button type="submit">Submit</button>
    </form>
    <br>
    <a href="/dashboard">Go to Dashboard</a>
</body>
</html>


## Step 4: Create the template for the dashboard page

Create a file named dashboard.html in the templates directory with the following content:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Daily Activity Dashboard</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h1>Daily Activity Dashboard</h1>
    <img src="data:image/png;base64,{{ plot_url }}" alt="Seaborn Plot">
    <br>
    <a href="/">Back to Data Entry</a>
</body>
</html>


## Step 5: Create the CSS File

Create a file named style.css in the static directory with the following content:

In [ ]:
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f4;
    margin: 0;
    padding: 0;
    display: flex;
    flex-direction: column;
    align-items: center;
}

h1 {
    color: #333;
}

form {
    background: #fff;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
    margin-top: 20px;
}

label {
    display: block;
    margin-bottom: 8px;
    color: #555;
}

input[type="text"],
input[type="number"] {
    width: 100%;
    padding: 8px;
    margin-bottom: 10px;
    border: 1px solid #ccc;
    border-radius: 4px;
}

button {
    background-color: #28a745;
    color: white;
    padding: 10px 15px;
    border: none;
    border-radius: 4px;
    cursor: pointer;
}

button:hover {
    background-color: #218838;
}

a {
    color: #007bff;
    text-decoration: none;
    margin-top: 20px;
}

a:hover {
    text-decoration: underline;
}

img {
    max-width: 100%;
    height: auto;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
}


## Step 6: Run the Application

Run your Flask application by executing the following command in your project directory:

In [ ]:
python app.py

Open your web browser and navigate to http://127.0.0.1:5000/ to see the Seaborn plot displayed on the webpage.